## QUEST
- db_NHIS.NSC2_M20 in mongodb
- 연속형 타입 3가지 선택(target, label), csv 저장
- 전처리, 학습, 평가, 예측 과정 진행

In [11]:
import pymongo as mg
import pandas as pd

#### mogoDB에서 파일 SELECT

In [3]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['db_NHIS']
collection = database['NSC2_M20']
cursor = collection.find({})
list_M20 = list(cursor)
df_M20 = pd.DataFrame(list_M20)

In [4]:
df_M20[:2]

,_id,RN_INDI,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,MCARE_SUBJ_CD,SICK_SYM1,OFIJ_TYPE,OPRTN_YN,...,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY,HSPTZ_PATH_TYPE,SICK_SYM2,SPCF_SYM_TYPE,FST_HSPTZ_DT
0,64e5bba4cea655f0a39a0961,429577,2002070826167,101494,20020729,3,15,B07,0,0,...,5.0,0.15,10590,3000,7590,2002,NaN,NaN,NaN,NaN
1,64e5bba4cea655f0a39a0962,362517,2002070388352,82780,20020729,3,14,B354,NaN,0,...,5.0,0.15,10590,3000,7590,2002,NaN,NaN,NaN,NaN


In [7]:
df_M20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137163 entries, 0 to 137162
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   _id                 137163 non-null  object 
 1   RN_INDI             137163 non-null  int64  
 2   RN_KEY              137163 non-null  int64  
 3   RN_INST             137163 non-null  int64  
 4   MDCARE_STRT_DT      137163 non-null  int64  
 5   FORM_CD             137163 non-null  int64  
 6   MCARE_SUBJ_CD       137163 non-null  int64  
 7   SICK_SYM1           137163 non-null  object 
 8   OFIJ_TYPE           134428 non-null  object 
 9   OPRTN_YN            137163 non-null  int64  
 10  MDCARE_DD_CNT       137163 non-null  int64  
 11  VSHSP_DD_CNT        137163 non-null  int64  
 12  TOT_PRSC_DD_CNT     137163 non-null  int64  
 13  MCARE_RSLT_TYPE     133958 non-null  float64
 14  EDC_ADD_RT          137163 non-null  float64
 15  ED_RC_TOT_AMT       137163 non-nul

In [8]:
df_M20.columns

Index(['_id', 'RN_INDI', 'RN_KEY', 'RN_INST', 'MDCARE_STRT_DT', 'FORM_CD',
       'MCARE_SUBJ_CD', 'SICK_SYM1', 'OFIJ_TYPE', 'OPRTN_YN', 'MDCARE_DD_CNT',
       'VSHSP_DD_CNT', 'TOT_PRSC_DD_CNT', 'MCARE_RSLT_TYPE', 'EDC_ADD_RT',
       'ED_RC_TOT_AMT', 'EDC_SBA', 'EDC_INSUR_BRDN_AMT', 'STD_YYYY',
       'HSPTZ_PATH_TYPE', 'SICK_SYM2', 'SPCF_SYM_TYPE', 'FST_HSPTZ_DT'],
      dtype='object')

In [9]:
df_M20 = df_M20[['MDCARE_STRT_DT','VSHSP_DD_CNT','EDC_INSUR_BRDN_AMT']] 

In [10]:
df_M20.to_csv('../../datasets/NSC2_M20.csv')

#### 추출한 컬럼들을 토대로 지도학습 진행 

## 지도학습(Supervised Learning)
- 목표변수(target, Y) 존재하는 학습법 
- 설명변수 

In [12]:
df_M20 = pd.read_csv('../../datasets/NSC2_M20.csv')
df_M20[:2]

,Unnamed: 0,MDCARE_STRT_DT,VSHSP_DD_CNT,EDC_INSUR_BRDN_AMT
0,0,20020729,1,7590
1,1,20020729,1,7590


### 목표변수 - 연속형
- 목표변수 : MDCARE_STRT_DT
- 설명변수 : VSHSP_DD_CNT, EDC_INSUR_BRDN_AMT

In [29]:
df_M20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137163 entries, 0 to 137162
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype
---  ------              --------------   -----
 0   Unnamed: 0          137163 non-null  int64
 1   MDCARE_STRT_DT      137163 non-null  int64
 2   VSHSP_DD_CNT        137163 non-null  int64
 3   EDC_INSUR_BRDN_AMT  137163 non-null  int64
dtypes: int64(4)
memory usage: 4.2 MB


### Pre-Processing (전처리과정)

##### 목표변수와 설명변수 추출

In [15]:
df_M20.isnull().sum()

Unnamed: 0            0
MDCARE_STRT_DT        0
VSHSP_DD_CNT          0
EDC_INSUR_BRDN_AMT    0
dtype: int64

##### Structured data 
- 목표변수(**target**, Y)
- 설명변수(**label**, X)
- 목표변수와 설명변수 분리 

In [16]:
target = df_M20['MDCARE_STRT_DT']
labels = df_M20[['VSHSP_DD_CNT','EDC_INSUR_BRDN_AMT']]
target.shape, labels.shape

((137163,), (137163, 2))

### 모델(알고리즘) 학습

#### 모델 학습 
- target datatype 종속적 

In [17]:
from sklearn.linear_model import LinearRegression

In [18]:
model = LinearRegression()
model.fit(labels, target)

LinearRegression()

#### 평가

In [21]:
model.predict(labels) ## 나온 결과값이 target의 값

array([20099489.74079853, 20099489.74079853, 20099489.74079853, ...,
       20085474.05584997, 20073522.7100947 , 20028511.97992517])

In [23]:
target_predict = model.predict(labels) 
len(target_predict)

137163

In [24]:
from sklearn.metrics import r2_score
r2_score(target, target_predict)  ## 값을 뽑기위한 예측 

0.019125218064909077

#### 미래예측(서비스 개시)

In [25]:
df_M20[100:105]

,Unnamed: 0,MDCARE_STRT_DT,VSHSP_DD_CNT,EDC_INSUR_BRDN_AMT
100,100,20020419,1,5160
101,101,20020419,1,5160
102,102,20020724,1,5160
103,103,20020730,1,5160
104,104,20020622,1,5160


In [26]:
# 1, 5160 : 예측하기 위해 임의의 값을 입력함
temp_label = [[1, 5160]]

In [31]:
model.predict(temp_label)

## 실제값 : 20020419 / 예측의 값 : 20099457.5422
## 연속형인 데이터 자체가 실제로는 진료코드여서 예측의 의미가 없다는 것을 확인함. 

C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([20099457.54221297])